# Diabetic retinopathy detection using Convolutional Neural Networks

In this notebook I'll show how to use CNNs and transfer learning to train a diabetic retinopathy detection model using fundus images.

---

## *Detecção de retinopatia diabética utilizando redes neurais convolucionais*

*Neste notebook irei demonstrar como utilizar redes neurais convolucionais e transferência de aprendizado para treinar um modelo de detecção de retinopatia diabética a partir de imagens de retina.*

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from os import listdir
from os.path import isfile, join
import os

import tensorflow as tf # google's library for deep learning
from tensorflow.keras.utils import Sequence, to_categorical
import cv2 # opencv - for computer vision
from tensorflow.keras.preprocessing.image import (ImageDataGenerator, load_img,
                                                  save_img)

from tensorflow.keras.layers import (AveragePooling2D, Conv2D, Dense, Dropout,
                                     Flatten, Input, Lambda, LeakyReLU,
                                     MaxPooling2D, UpSampling2D)
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
from sklearn.utils import shuffle
import math
from typing import AnyStr, Callable


## List all image files and its DR levels on a DataFrame

The used dataset can be found [here](https://www.kaggle.com/sovitrath/diabetic-retinopathy-2015-data-colored-resized).

---

### *Lista todos arquivos de imagem e seus níveis de RD e os coloca num DataFrame*

O banco de dados utilizado pode ser encontrado [aqui](https://www.kaggle.com/sovitrath/diabetic-retinopathy-2015-data-colored-resized).

In [ ]:
df = pd.DataFrame(columns=['image','file','level'])
mypath = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/No_DR/'
df['image'] = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df['level'] = 0
df['file'] = mypath + df['image']

df2 = pd.DataFrame(columns=['image','file','level'])
mypath = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Mild/'
df2['image'] = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df2['level'] = 1
df2['file'] = mypath + df2['image']
df = df.append(df2,ignore_index=True)

df2 = pd.DataFrame(columns=['image','file','level'])
mypath = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Moderate/'
df2['image'] = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df2['level'] = 2
df2['file'] = mypath + df2['image']
df = df.append(df2,ignore_index=True)

df2 = pd.DataFrame(columns=['image','file','level'])
mypath = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Severe/'
df2['image'] = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df2['level'] = 3
df2['file'] = mypath + df2['image']
df = df.append(df2,ignore_index=True)

df2 = pd.DataFrame(columns=['image','file','level'])
mypath = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images/Proliferate_DR/'
df2['image'] = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df2['level'] = 4
df2['file'] = mypath + df2['image']
df = df.append(df2,ignore_index=True)

df.head()

The DataFrame df contains 3 columns:
- image: the name of the each image.
- file: the complete path of each image file.
- level: the DR level.
---

*O DataFrame df contém 3 colunas:*
- *image: o nome de cada imagem.*
- *file: o caminho completo de cada arquivo de imagem.*
- *level: o nível de RD de cada imagem.*

### Define the Sequence, structure responsible for providing the model with preprocessed data

---

### *Define a estrutura Sequence, responsável por prover as amostras ao model durante o treinamento*

In [ ]:
class RetinaSequence(Sequence):

    def __init__(self, x_set: list, y_set: list, batch_size: int = 32, augmentate: bool = True, 
                 shuffle: bool = True, input_shape: tuple = (512, 512), preprocessing_fcn: callable = None):
        """ 
        This is called when the object is initialized.
        
        Parameters
        ----------
        x_set : list
            The array with image files paths.
        y_set : list
            The array with DR levels.  
        batch_size : int
            The batch size, which is how many samples are delivered at each update of the training algorithm.
        augmentate : bool
            If True, will use data augmentation.
        shuffle: bool
            If True, will shuffle the data at the end of each epoch.
        input_shape: tuple
            The size of the input images. Larger input images needs GPUs with more memory.
        preprocessing_fcn: callable
            Method for the preprocessing function that will be applied to the input images.
            
        print_cols : bool, optional
            A flag used to print the columns to the console (default is False)

        
        """
        
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.augmentate = augmentate
        self.shuffle = shuffle
        self.preprocessing_fcn = preprocessing_fcn
        self.input_shape = input_shape

        self.datagen = ImageDataGenerator(featurewise_center=False,
                                          samplewise_center=False,
                                          featurewise_std_normalization=False,
                                          samplewise_std_normalization=False,
                                          rotation_range=360.,
                                          width_shift_range=0.0,
                                          height_shift_range=0.0,
                                          shear_range=0.,
                                          zoom_range=0.0,
                                          channel_shift_range=0.,
                                          fill_mode='constant',
                                          cval=0.,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          rescale=None,
                                          preprocessing_function=None,
                                          data_format=K.image_data_format())

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        """This is the method called to actually give data to the model.

        Args:
            idx (int): index.

        Returns:
            [ndarray]: array with a batch of data and its labels.
        """        
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([self.preprocess(file_name)
                         for file_name in batch_x]), np.array(batch_y)

    def preprocess(self, file_name):
        """The preprocessing method. It does the contrast enhancement. Data augmentation if asked and transform to float32.

        Args:
            file_name (str): the file path.

        Returns:
            ndarray: the preprocessed image.
        """        
        img = load_img(file_name, target_size=self.input_shape)
        x = np.array(img, dtype=np.uint8)
        alfa = 4.0
        tal = -4.0
        xf = cv2.GaussianBlur(
            x, ksize=(0, 0), sigmaX=img.size[0] // 30, sigmaY=0)
        xf = xf.astype(np.float32)
        x = x.astype(np.float32)
        x = alfa * x + tal * xf + 128.0
        x[x < 0] = 0
        x[x > 255] = 255

        if self.augmentate:
            x = self.datagen.random_transform(x)
            factor = np.random.uniform(low=0.6, high=1.67)
            x = np.array(x, dtype=K.floatx())
            x = 128 + factor * (x - 128)
            x[x > 255] = 255
            x[x < 0] = 0
        else:
            x = x.astype(np.float32)
        if self.preprocessing_fcn is None:
            x -= 127.0
            x /= 127.0
        else:
            x = x.astype(np.uint8)
            x = self.preprocessing_fcn(x)
        return x

    def on_epoch_end(self):
        """This is called on each epoch end and shuffles the data if self.shuffe is True.
        """
        if self.shuffle:
            self.x, self.y = shuffle(self.x, self.y)    

In [ ]:
test_ratio = 0.5 # this is the percentage of data that will be used as test set, where we'll verify the model's performance.
validation_ratio = 0.2 # this is the percentage of data that will be used as validation set, used for early stopping.

df['class'] = df.level > 1
n = df.shape[0]
weights = {False: n / df.loc[df['class'] == False].shape[0],
           True: n / df.loc[df['class'] == True].shape[0]}
df['weights'] = df['class'].map(weights) # higher weights for class with less examples.

images = df.image.unique()

msk = np.random.rand(len(df)) < (1.0 - test_ratio)
train = df.loc[df.image.isin(images[msk])]
test = df.loc[df.image.isin(images[~msk])]

images = train.image.unique()
msk = np.random.rand(len(train)) < (1.0 - validation_ratio)
val = train.loc[train.image.isin(images[~msk])]
train = train.loc[train.image.isin(images[msk])]

train = train.sample(n=4 * len(train.index),
                     replace=True, weights='weights') # balancing the classes in the set used for training.

val = val.sample(n=4 * len(val.index),
                     replace=True, weights='weights') # balancing the classes in the set used for validation.

nb_train = len(train.index)
nb_val = len(val.index)

In [ ]:
preprocessing_fcn = tf.keras.applications.nasnet.preprocess_input # the preprocessing function is the default for the used model.

# we are going to use NASNetMobile because it is small and fast to train.
base_model = tf.keras.applications.nasnet.NASNetMobile(include_top=False, weights='imagenet',
                                                    input_tensor=None,
                                                    pooling='avg')
input_shape = base_model.input_shape[1:3]

# Freeze the pretrained weights
base_model.trainable = False

# adding the final part of the model.
x = base_model.output
x = Dense(units=512, activation='relu', use_bias=True, kernel_regularizer=tf.keras.regularizers.l2(0.005),
          bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dense(units=256, activation='relu', use_bias=True, kernel_regularizer=tf.keras.regularizers.l2(0.005),
          bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dense(units=2, activation='softmax', use_bias=True, kernel_regularizer=tf.keras.regularizers.l2(0.005),
          bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(x)
model = tf.keras.Model(inputs=base_model.input,
                    outputs=x, name='NASNetMobile')

In [ ]:
# Instantiate the sequences.
train_seq = RetinaSequence(x_set=train['file'],
                           y_set=to_categorical(
                               train['class'], num_classes=2),
                           batch_size=16, augmentate=True, shuffle=True,
                           input_shape=input_shape,
                           preprocessing_fcn=preprocessing_fcn)
val_seq = RetinaSequence(x_set=val['file'], y_set=to_categorical(val['class'], num_classes=2),
                         batch_size=16, augmentate=False, shuffle=False,
                         input_shape=input_shape,
                         preprocessing_fcn=preprocessing_fcn)

First we will train the recently added layers for a couple of epochs.

In [ ]:
# those are the metrics we are going to check.
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=METRICS)

epochs = 5
model.fit(
    x=train_seq,
    steps_per_epoch=nb_train // (4 * 16),
    epochs=epochs,
    verbose=1,
    validation_data=val_seq,
    validation_steps=nb_val // (4*16),
    max_queue_size=10,
    workers=20,
    use_multiprocessing=False,
    shuffle=False)

Then, we unfreeze the layers of the pre-trained model and train it all.

In [ ]:
patience_early_stop = 5
for layer in model.layers:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# compile the model using Stochastic Gradient Descent optimization algorithm.
model.compile(optimizer=optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True),
              loss='binary_crossentropy', metrics=METRICS)

# The callbacks are the methods called during the training process.
# EarlyStopping will stop the training process if the loss stops getting better.

# ModelCheckpoint will save the model at the best epoch.
callbacks = [tf.keras.callbacks.EarlyStopping(
                 monitor='val_loss',
                 patience=patience_early_stop),
                 tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/model_DR.h5', monitor="val_loss", verbose=1, save_best_only=True, save_weights_only=False, mode="auto", save_freq="epoch")
                     ]

model.fit(
    x=train_seq,
    steps_per_epoch=nb_train // (4 * 16),
    epochs=50,
    verbose=1,
    callbacks=callbacks,
    validation_data=val_seq,
    validation_steps= nb_val // 16,
    max_queue_size=10,
    workers=20,
    use_multiprocessing=False,
    shuffle=False)

## Now let's evaluate our model.
---
## Agora vamos avaliar nosso modelo.

In [ ]:
test_seq = RetinaSequence(x_set=test['file'], y_set=to_categorical(test['class'], num_classes=2),
                         batch_size=16, augmentate=False, shuffle=False,
                         input_shape=input_shape,
                         preprocessing_fcn=preprocessing_fcn)
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_seq)
print("test loss, test acc:", results)